In [5]:
%%time

from functools import reduce
from pyspark.sql import DataFrame

# df1 = spark.read.format('json').load("hdfs://orion11:33001/reddit/2013/*")
# df2 = spark.read.format('json').load("hdfs://orion11:33001/reddit/2014/*")
# df3 = spark.read.format('json').load("hdfs://orion11:33001/reddit/2015/*")
# df4 = spark.read.format('json').load("hdfs://orion11:33001/reddit/2016/*")
# df5 = spark.read.format('json').load("hdfs://orion11:33001/reddit/2017/*")

df = spark.read.json(['hdfs://orion11:33001/reddit/2013','hdfs://orion11:33001/reddit/2014','hdfs://orion11:33001/reddit/2015', 'hdfs://orion11:33001/reddit/2016', 'hdfs://orion11:33001/reddit/2017'])



CPU times: user 927 ms, sys: 573 ms, total: 1.5 s
Wall time: 1h 56min 30s


In [6]:
df.count()

2630956849

In [7]:
samp = df.sample(False, .05)
# Write it out
samp.write.format('json').save('hdfs://orion11:33001/warmup3')

In [8]:
df_samp = spark.read.format('json').load("hdfs://orion11:33001/warmup3/*")

In [9]:
df_samp.show()

+--------+------------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+----------+------+-------+---------+-----------+----------+----------+-------+------------+-----+-----+------------+--------+--------------+------------+---+------------+
|archived|            author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|    edited|gilded|     id|  link_id|mod_reports|      name| parent_id|replies|retrieved_on|saved|score|score_hidden|stickied|     subreddit|subreddit_id|ups|user_reports|
+--------+------------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+----------+------+-------+---------+-----------+----------+----------+-------+------------+-----+-----+------------+--------+--------------

In [10]:
from pyspark.sql.functions import year, month, dayofmonth, from_unixtime
from pyspark.sql.types import TimestampType, DateType

botExpr = "[bB][oO][tT]"

df_samp2 = (df_samp
                .filter(~(df_samp.body.like("[deleted]") | df_samp.author.rlike(botExpr)))
                .withColumn("day", dayofmonth(from_unixtime("created_utc").cast(DateType())))
                .withColumn("month", month(from_unixtime("created_utc").cast(DateType()))))
df_samp2.select("body", "created_utc", "day", "month").show()

+--------------------+-----------+---+-----+
|                body|created_utc|day|month|
+--------------------+-----------+---+-----+
|Alright so I just...| 1364774402| 31|    3|
|Or if the have di...| 1364774403| 31|    3|
|If I were to gues...| 1364774403| 31|    3|
|My big thing woul...| 1364774403| 31|    3|
|Absolutely. This ...| 1364774404| 31|    3|
|Don't think it's ...| 1364774405| 31|    3|
|If I'm paying the...| 1364774406| 31|    3|
|            *shoots*| 1364774408| 31|    3|
|Thanks, I'll try ...| 1364774408| 31|    3|
|Very interested i...| 1364774409| 31|    3|
|I was being a bit...| 1364774411| 31|    3|
|True. TISP took m...| 1364774412| 31|    3|
|[Husky made one w...| 1364774415| 31|    3|
|That's how I wish...| 1364774417| 31|    3|
|Anybody know why ...| 1364774422| 31|    3|
|The transparent h...| 1364774422| 31|    3|
|You don't combine...| 1364774424| 31|    3|
|Actualllyyy. Tech...| 1364774426| 31|    3|
|I think it's part...| 1364774427| 31|    3|
|It's diff

In [11]:
df_samp3 = df_samp2.filter((df_samp2.day == 9) & (df_samp2.month == 11))

In [12]:
df_samp3.select("body", "created_utc", "day", "month").show()

+--------------------+-----------+---+-----+
|                body|created_utc|day|month|
+--------------------+-----------+---+-----+
|Fairy Tale Advent...| 1383984002|  9|   11|
|             Asians.| 1383984003|  9|   11|
|Wasting 90% of my...| 1383984004|  9|   11|
|&gt; I think he t...| 1383984005|  9|   11|
|What happens to m...| 1383984006|  9|   11|
|Commenting for la...| 1383984006|  9|   11|
|No need to apolog...| 1383984007|  9|   11|
|Oh no, I didn't m...| 1383984013|  9|   11|
|Authoritarianism....| 1383984013|  9|   11|
|Yeah, the CD rele...| 1383984016|  9|   11|
|[Wisconsin Death ...| 1383984019|  9|   11|
|Do you take amazo...| 1383984020|  9|   11|
|Jesus was a filth...| 1383984022|  9|   11|
|Crunchbang is pla...| 1383984023|  9|   11|
|*shakes it* Pleas...| 1383984025|  9|   11|
|On the contrary, ...| 1383984028|  9|   11|
|The monitor doesn...| 1383984032|  9|   11|
|        flip a coin
| 1383984032|  9|   11|
|Interesting. Do y...| 1383984033|  9|   11|
|No, it is

In [13]:
df_samp3.count()

342393